## Imports

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.optimizers.schedules import ExponentialDecay  # from https://arxiv.org/pdf/1506.02078.pdf
from tensorflow.keras.callbacks import EarlyStopping

from tqdm.notebook import tqdm

In [2]:
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  0


## Hyper-parameters

In [3]:
tunable_hparams = {
    'stateful_generation' : True,
    'mapping_type' : 'seq2seq',
    'early_stopping' : False,
    'seq_length' : 200,
    'game' : 'mario'
}
fixed_hparams = {
    'hidden_size' : 128,
    'learning_rate' : 2e-3,
    'learning_rate_decay' : 0.95,
    'dropout' : 0.5,
    'batch_size' : 100,
    'num_layers' : 3,
    'max_epochs' : 50
}

In [4]:
for key, val in tunable_hparams.items():
    exec(key + '=val')
for key, val in fixed_hparams.items():
    exec(key + '=val')

## Data

In [10]:
data = open('corpuses/mario_corpus_conditional.txt', 'r').read()
level_strs = data.rstrip().split(')')[:-1]
print(len(level_strs))

37


In [11]:
chars = []
for level_str in level_strs:
    chars.extend(list(level_str))
chars = list(set(chars))
vocab_size = len(chars)
print(chars, vocab_size)

['\n', '[', 'E', '>', 'S', 'B', 'Q', '<', 'b', '-', 'X', 'Y', 'x', '?', 'N', ']', 'o'] 17


In [12]:
char_to_ix = { ch:i for i, ch in enumerate(chars) }
ix_to_char = { i:ch for i, ch in enumerate(chars) }

In [13]:
ix_to_char

{0: '\n',
 1: '[',
 2: 'E',
 3: '>',
 4: 'S',
 5: 'B',
 6: 'Q',
 7: '<',
 8: 'b',
 9: '-',
 10: 'X',
 11: 'Y',
 12: 'x',
 13: '?',
 14: 'N',
 15: ']',
 16: 'o'}

In [14]:
level_arrays = []
for level_str in level_strs:
    level_arrays.append(np.array([char_to_ix[char] for char in list(level_str)]))

In [15]:
def get_inputs_and_targets_from_level_array(level_array):
    
    inputs, targets = [], []
    
    for i in range(len(level_array) - seq_length):
        inputs.append(level_array[i:i+seq_length])
        targets.append(level_array[i+1:i+seq_length+1])
    
    inputs, targets = map(np.array, [inputs, targets])
    inputs = np.eye(vocab_size)[inputs]
    
    return inputs, targets

In [16]:
inputs, targets = [], []
for level_array in tqdm(level_arrays, leave=False):
    inputs_temp, targets_temp = get_inputs_and_targets_from_level_array(level_array)
    inputs.extend(inputs_temp); targets.extend(targets_temp)
inputs, targets = map(np.array, [inputs, targets])

In [17]:
inputs.shape, targets.shape

((156360, 200, 17), (156360, 200))

## Model callbacks

In [19]:
lr_scheduler = ExponentialDecay(
    initial_learning_rate=learning_rate,
    decay_steps=len(inputs) // batch_size, 
    decay_rate=learning_rate_decay, 
)
optimizer = RMSprop(learning_rate=lr_scheduler)

In [20]:
es_callback = EarlyStopping(
    monitor='val_out_acc_custom_acc', mode='max', patience=2, restore_best_weights=early_stopping
)

In [21]:
def custom_loss(y_true, y_pred):
    scce = tf.keras.losses.SparseCategoricalCrossentropy()
    return scce(
        tf.reshape(y_true, shape=(tf.shape(y_true)[0] * seq_length, )), 
        tf.reshape(y_pred, shape=(tf.shape(y_pred)[0] * seq_length, vocab_size))
    )

In [22]:
def custom_acc(y_true, y_pred):
    return tf.math.reduce_mean(
        tf.cast(
            tf.math.equal(
                tf.math.argmax(tf.reshape(y_pred, shape=(tf.shape(y_pred)[0] * seq_length, vocab_size)), axis=-1), 
                tf.cast(tf.reshape(y_true, shape=(tf.shape(y_true)[0] * seq_length, )), dtype=tf.int64)
            ), 
            dtype=tf.float32
        )
    )

## Model definition

In [16]:
lstm_1_state_h_in = keras.layers.Input(shape=[hidden_size])
lstm_1_state_c_in = keras.layers.Input(shape=[hidden_size])

lstm_2_state_h_in = keras.layers.Input(shape=[hidden_size])
lstm_2_state_c_in = keras.layers.Input(shape=[hidden_size])

lstm_3_state_h_in = keras.layers.Input(shape=[hidden_size])
lstm_3_state_c_in = keras.layers.Input(shape=[hidden_size])

input = keras.layers.Input(shape=[seq_length, vocab_size])

out, lstm_1_state_h_out, lstm_1_state_c_out = keras.layers.LSTM(hidden_size, return_sequences=True, return_state=True)(
    input, initial_state=[lstm_1_state_h_in, lstm_1_state_c_in]
)
out = layers.Dropout(dropout)(out)

out, lstm_2_state_h_out, lstm_2_state_c_out = keras.layers.LSTM(hidden_size, return_sequences=True, return_state=True)(
    out, initial_state=[lstm_2_state_h_in, lstm_2_state_c_in]
)
out = layers.Dropout(dropout)(out)

out, lstm_3_state_h_out, lstm_3_state_c_out = keras.layers.LSTM(hidden_size, return_sequences=True, return_state=True)(
    out, initial_state=[lstm_3_state_h_in, lstm_3_state_c_in]
)
out = layers.Dropout(dropout)(out)

out = layers.Dense(vocab_size)(out)
out = layers.Activation('softmax')(out)

out_acc = layers.Lambda(lambda x:x, name = "out_acc")(out)

model = keras.models.Model(
    inputs=[
        input, 
        lstm_1_state_h_in, lstm_1_state_c_in,
        lstm_2_state_h_in, lstm_2_state_c_in,
        lstm_3_state_h_in, lstm_3_state_c_in
    ], 
    outputs=[
        out_acc,
        lstm_1_state_h_out, lstm_1_state_c_out,
        lstm_2_state_h_out, lstm_2_state_c_out,
        lstm_3_state_h_out, lstm_3_state_c_out
    ]
)

model.compile(
    loss=[custom_loss, None, None, None, None, None, None], 
    loss_weights=[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
    metrics={'out_acc':custom_acc},
    optimizer=optimizer
)

## Model training

In [17]:
dummy = np.zeros((len(inputs), hidden_size))

In [18]:
history = model.fit(
    [inputs, dummy, dummy, dummy, dummy, dummy, dummy], 
    [targets, dummy, dummy, dummy, dummy, dummy, dummy], 
    batch_size=batch_size,
    validation_split=0.1,
    shuffle=True,
    epochs=max_epochs, 
    callbacks=[es_callback]
)
for i in range(10):
    model.save('lstm_conditional.h5')

Epoch 1/50
1140/1140 [==============================] - 158s 138ms/step - loss: 0.4111 - out_acc_loss: 0.4111 - out_acc_custom_acc: 0.8785 - val_loss: 0.3668 - val_out_acc_loss: 0.3668 - val_out_acc_custom_acc: 0.8844
Epoch 2/50
1140/1140 [==============================] - 156s 137ms/step - loss: 0.1537 - out_acc_loss: 0.1537 - out_acc_custom_acc: 0.9533 - val_loss: 0.3750 - val_out_acc_loss: 0.3750 - val_out_acc_custom_acc: 0.9099
Epoch 4/50
1140/1140 [==============================] - 156s 137ms/step - loss: 0.1356 - out_acc_loss: 0.1356 - out_acc_custom_acc: 0.9586 - val_loss: 0.4130 - val_out_acc_loss: 0.4130 - val_out_acc_custom_acc: 0.9105


## Load trained model

In [25]:
model = keras.models.load_model(
    'trained_models/lstm_conditional_elements.h5', 
    custom_objects={'custom_loss':custom_loss, 'custom_acc':custom_acc}
)

In [20]:
model.evaluate(
    [inputs, dummy, dummy, dummy, dummy, dummy, dummy], 
    [targets, dummy, dummy, dummy, dummy, dummy, dummy],
    batch_size=5, verbose=1
)  # sanity check

  151/25319 [..............................] - ETA: 20:55 - loss: 0.1233 - out_acc_loss: 0.1233 - out_acc_custom_acc: 0.9629

KeyboardInterrupt: 

## Generate level

In [26]:
def onehot_to_string(onehot):
    ints = np.argmax(onehot, axis=-1)
    chars = [ix_to_char[ix] for ix in ints]
    string = "".join(chars)
    char_array = []
    if len(string.rstrip().split('\n')[-1]) < 17:
        for line in string.rstrip().split('\n')[:-1]:
            char_array.append(list(line))
    else:
        for line in string.rstrip().split('\n'):
            char_array.append(list(line))
    char_array = np.array(char_array).T
    string = ""
    for row in char_array:
        string += "".join(row) + "\n"
    return string

In [28]:
seed = inputs[0][:3 * 18 - 2].copy()  # 3 cols * 18 tiles per col - newline char - condition char
seed[17+16] = 0
seed[17+16][12] = 1
seed[17*2+17] = 0
seed[17*2+17][12] = 1
print(seed.shape)
print(onehot_to_string(seed))

(52, 17)
NN
NN
NN
NN
NN
--
--
--
--
--
--
-x
--
--
--
--
--
--
--
--
XX



In [76]:
num_chunks_to_gen = 5# just for testing purposed
num_tile_to_gen = 1 + num_chunks_to_gen * 16 * 17  # 1 newline char for the 3rd col, condition char are offered directly

In [77]:
condition_tape_question = [0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0] * num_chunks_to_gen
condition_tape_coin     = [1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0] * num_chunks_to_gen
condition_tape_enemy    = [0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0] * num_chunks_to_gen
condition_tape_pipe     = [0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0] * num_chunks_to_gen
condition_tape_cannon   = [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1] * num_chunks_to_gen

print(len(condition_tape_question))  # should be of length 16

80


In [78]:
for j in tqdm(range(1, 20+1)):

    seed = inputs[0][:3 * 18 - 2].copy()  # 3 cols * 18 tiles per col - newline char - condition char
    seed[17+16] = 0
    seed[17+16][10] = 1
    seed[17*2+17] = 0
    seed[17*2+17][10] = 1
    gen = seed.copy()

    # initialize all hidden and cell states to zeros
    lstm1_h = np.zeros((1, hidden_size))
    lstm1_c = np.zeros((1, hidden_size))
    lstm2_h = np.zeros((1, hidden_size))
    lstm2_c = np.zeros((1, hidden_size))
    lstm3_h = np.zeros((1, hidden_size))
    lstm3_c = np.zeros((1, hidden_size))
    
    add_condition_char_next = False
    col_ix_generating = -1

    for i in tqdm(range(num_tile_to_gen), leave=False):

        seed = np.expand_dims(seed, axis=0)

        # predict probas and update hidden and cell states
        probas, lstm1_h, lstm1_c, lstm2_h, lstm2_c, lstm3_h, lstm3_c = model.predict([
            seed, lstm1_h, lstm1_c, lstm2_h, lstm2_c, lstm3_h, lstm3_c
        ])
        
        # ========== generic prediction ==========
        
        if not add_condition_char_next:

            probas = probas[0][-1]  # first batch, last timestep

            idx = np.random.choice(np.arange(len(probas)), p=probas)
            seed = np.zeros((1, vocab_size))
            seed[:, idx] = 1.
            gen = np.vstack([gen, seed])
            
            if ix_to_char[idx] == '\n':
                add_condition_char_next = True
                col_ix_generating += 1
            
        # ========== condition char are not generated, they are loaded from the condition tape ==========
            
        else:
            
            seed = np.zeros((1, vocab_size))
            if condition_tape[col_ix_generating] == 0:
                seed[:, char_to_ix['N']] = 1
            elif condition_tape[col_ix_generating] == 1:
                seed[:, char_to_ix['Y']] = 1
            gen = np.vstack([gen, seed])
            
            add_condition_char_next = False
        
    with open(f'./lstm_conditional_generated_levels_txt/{j}.txt', 'w+') as txt_f:
        txt_f.write(onehot_to_string(gen))

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

